In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler

spark = SparkSession.builder \
    .appName("LogisticRegression with PySpark MLlib") \
    .getOrCreate()

24/09/27 15:19:25 WARN Utils: Your hostname, AI-CJB-LAP-459 resolves to a loopback address: 127.0.1.1; using 192.168.1.164 instead (on interface wlp0s20f3)
24/09/27 15:19:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/27 15:19:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
# from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType

# Create a Spark session
# spark = SparkSession.builder.appName("BreastCancerData").getOrCreate()

# Define the schema excluding the _c32 column
schema = StructType([
    StructField("id", StringType(), True),
    StructField("diagnosis", StringType(), True),
    StructField("radius_mean", FloatType(), True),
    StructField("texture_mean", FloatType(), True),
    StructField("perimeter_mean", FloatType(), True),
    StructField("area_mean", FloatType(), True),
    StructField("smoothness_mean", FloatType(), True),
    StructField("compactness_mean", FloatType(), True),
    StructField("concavity_mean", FloatType(), True),
    StructField("concave points_mean", FloatType(), True),
    StructField("symmetry_mean", FloatType(), True),
    StructField("fractal_dimension_mean", FloatType(), True),
    StructField("radius_se", FloatType(), True),
    StructField("texture_se", FloatType(), True),
    StructField("perimeter_se", FloatType(), True),
    StructField("area_se", FloatType(), True),
    StructField("smoothness_se", FloatType(), True),
    StructField("compactness_se", FloatType(), True),
    StructField("concavity_se", FloatType(), True),
    StructField("concave points_se", FloatType(), True),
    StructField("symmetry_se", FloatType(), True),
    StructField("fractal_dimension_se", FloatType(), True),
    StructField("radius_worst", FloatType(), True),
    StructField("texture_worst", FloatType(), True),
    StructField("perimeter_worst", FloatType(), True),
    StructField("area_worst", FloatType(), True),
    StructField("smoothness_worst", FloatType(), True),
    StructField("compactness_worst", FloatType(), True),
    StructField("concavity_worst", FloatType(), True),
    StructField("concave points_worst", FloatType(), True),
    StructField("symmetry_worst", FloatType(), True),
    StructField("fractal_dimension_worst", FloatType(), True),
    # If there is an extra column without a header, you can add it here:
    StructField("_c32", StringType(), True)  # Add or adjust this line based on your inspection
])


In [14]:
df = spark.read.csv("https://raw.githubusercontent.com/pkmklong/Breast-Cancer-Wisconsin-Diagnostic-DataSet/master/data.csv", header=True, schema=schema)
df.show()


UnsupportedOperationException: None

In [13]:
url = "https://raw.githubusercontent.com/pkmklong/Breast-Cancer-Wisconsin-Diagnostic-DataSet/master/data.csv"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("data.csv"), inferSchema=True)
df.show(2)
df.printSchema()

24/09/27 15:36:32 WARN SparkContext: The path https://raw.githubusercontent.com/pkmklong/Breast-Cancer-Wisconsin-Diagnostic-DataSet/master/data.csv has been added already. Overwriting of added paths is not supported in the current version.


+------+---------+-----------+------------+--------------+---------+---------------+----------------+--------------+-------------------+-------------+--------------------+---------+----------+------------+-------+-------------+--------------+------------+-----------------+-----------+--------------------+------------+-------------+---------------+----------+----------------+-----------------+---------------+--------------------+--------------+--------------------+----+
|   _c0|      _c1|        _c2|         _c3|           _c4|      _c5|            _c6|             _c7|           _c8|                _c9|         _c10|                _c11|     _c12|      _c13|        _c14|   _c15|         _c16|          _c17|        _c18|             _c19|       _c20|                _c21|        _c22|         _c23|           _c24|      _c25|            _c26|             _c27|           _c28|                _c29|          _c30|                _c31|_c32|
+------+---------+-----------+------------+---------